In [17]:
import pandas as pd

### Venta

In [18]:
# Ubicación archivos
URL_2 = 'C:/Users/juan.vergara/OneDrive - Caren Repuestos Flotacentro/Documentos/Análisis/Segmentación Clientes/RFM-2022/'

# Carga de archivo eliminando la fila 0 (viene null)
df2021 = pd.read_table(URL_2+'FechaNotaRUT2021.txt', sep = ',', ).drop(0)
df2020 = pd.read_table(URL_2+'FechaNotaRUT2020.txt', sep = ';', ).drop(0)
df2019 = pd.read_table(URL_2+'FechaNotaRUT2019.txt', sep = ';', ).drop(0)

# Concatenación
frames = [df2019, df2020, df2021]
df_1 = pd.concat(frames)

# Limpiando los datos '-' del campo fecha
df_1 = df_1[df_1.Fecha != '-']

# Formato de fecha
df_1['Fecha'] = pd.to_datetime(df_1['Fecha'], format = '%d-%m-%Y')

#Ventas:

# 36 meses
df_1 = df_1[(df_1.Fecha >= '2019-01-01') & (df_1.Fecha <= '2021-12-31')]

In [19]:
df_1.head()

,Fecha,NotaVenta,RUT,Cliente,$ Venta
1,2019-01-30,0.0,88397100-0,Transportes Bello E Hijos Ltda.,28.992
2,2019-01-31,0.0,76754296-8,Fedex Express Chile Spa,55.463
3,2019-02-28,0.0,88397100-0,Transportes Bello E Hijos Ltda.,39.916
4,2019-03-30,0.0,88397100-0,Transportes Bello E Hijos Ltda.,57.984
5,2019-04-30,0.0,88397100-0,Transportes Bello E Hijos Ltda.,28.992


In [20]:
df_1.loc[df_1['RUT'] == '96882280-2']

,Fecha,NotaVenta,RUT,Cliente,$ Venta
17214,2019-01-23,7437084.0,96882280-2,Importadora Transagro S.A.,78.063
17544,2019-01-30,7437433.0,96882280-2,Importadora Transagro S.A.,285.674
28723,2019-09-05,7449583.0,96882280-2,Importadora Transagro S.A.,112.329
28980,2019-09-10,7449873.0,96882280-2,Importadora Transagro S.A.,47.489
33333,2019-12-19,7454669.0,96882280-2,Importadora Transagro S.A.,1.217.700
...,...,...,...,...,...
263906,2021-12-20,7004542.0,96882280-2,Importadora Transagro S.A.,22.743
266491,2021-12-22,60002376.0,96882280-2,Importadora Transagro S.A.,446.021
268843,2021-12-27,60002507.0,96882280-2,Importadora Transagro S.A.,315.994
269930,2021-12-29,7005014.0,96882280-2,Importadora Transagro S.A.,311.316


### Grupos de RUT

In [21]:
# RUT Grupos
df_rut = pd.read_excel(URL_2+'RUT_Grupos.xlsx')

# Separamos el campo RUT del df_1
df_1['Rut'] = df_1['RUT'].str.split('-').str[0]
df_1['Rut'] = df_1['Rut'].astype('int64')

# Left Join, para traer el RUT del grupo
df_1 = pd.merge(df_1, df_rut, on ='Rut', how = 'left')

# If para rellenar la columna de RUT de grupo
df_1.loc[df_1['Rut_Grupo'].isnull(), 'RUT_Final'] = df_1['Rut']
df_1.loc[df_1['Rut_Grupo'].notnull(), 'RUT_Final'] = df_1['Rut_Grupo']

# Limpiamos el df
df_1 = df_1.drop('RUT',1)
df_1 = df_1.drop('Rut',1)
df_1 = df_1.drop('grupo',1)
df_1 = df_1.drop('Rut_Grupo',1)

df_1['RUT'] = df_1['RUT_Final']
df_1 = df_1.drop('RUT_Final',1)

df_1.head()

,Fecha,NotaVenta,Cliente,$ Venta,RUT
0,2019-01-30,0.0,Transportes Bello E Hijos Ltda.,28.992,88397100.0
1,2019-01-31,0.0,Fedex Express Chile Spa,55.463,76754296.0
2,2019-02-28,0.0,Transportes Bello E Hijos Ltda.,39.916,88397100.0
3,2019-03-30,0.0,Transportes Bello E Hijos Ltda.,57.984,88397100.0
4,2019-04-30,0.0,Transportes Bello E Hijos Ltda.,28.992,88397100.0


### Ventas

In [22]:
# 36 meses

df_1['$ Venta'] = df_1['$ Venta'].str.replace('.', '')
df_1['$ Venta'] = df_1['$ Venta'].astype(int)

df_2 = df_1.groupby(['RUT']).agg({'$ Venta': 'sum'}).reset_index()
df_2.columns = ['RUT_Final','Venta_36_meses']

df_2.head()

<ipython-input-22-a052fed94866>:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df_1['$ Venta'] = df_1['$ Venta'].str.replace('.', '')


,RUT_Final,Venta_36_meses
0,148845.0,35545.0
1,308519.0,77160.0
2,530450.0,69144.0
3,693520.0,1593673.0
4,1062033.0,62261.0


In [23]:
# 24 meses

df_24 = df_1[(df_1.Fecha >= '2020-01-01') & (df_1.Fecha <= '2021-12-31')]

df_24_2 = df_24.groupby(['RUT']).agg({'$ Venta': 'sum'}).reset_index()
df_24_2.columns = ['RUT_Final','Venta_24_meses']

df_24_2.head()

,RUT_Final,Venta_24_meses
0,148845.0,35545.0
1,308519.0,65874.0
2,530450.0,69144.0
3,693520.0,1180560.0
4,1062033.0,62261.0


In [24]:
# 12 meses

df_12 = df_1[(df_1.Fecha >= '2021-01-01') & (df_1.Fecha <= '2021-12-31')]

df_12_2 = df_12.groupby(['RUT']).agg({'$ Venta': 'sum'}).reset_index()
df_12_2.columns = ['RUT_Final','Venta_12_meses']

df_12_2.head()

,RUT_Final,Venta_12_meses
0,308519.0,65874
1,530450.0,69144
2,693520.0,1167745
3,1062033.0,62261
4,1226290.0,114946


In [25]:
# Left Join

URL_D = 'C:/Users/juan.vergara/OneDrive - Caren Repuestos Flotacentro/Documentos/Análisis/Segmentación Clientes/Segmentación de Clientes para Modelo/'

df_2.to_csv(URL_D + 'Ventas_36_meses.csv', index = False)

df_24_2.to_csv(URL_D + 'Ventas_24_meses.csv', index = False)

df_12_2.to_csv(URL_D + 'Ventas_12_meses.csv', index = False)

In [26]:
# Nombres

df_1.head()

,Fecha,NotaVenta,Cliente,$ Venta,RUT
0,2019-01-30,0.0,Transportes Bello E Hijos Ltda.,28992,88397100.0
1,2019-01-31,0.0,Fedex Express Chile Spa,55463,76754296.0
2,2019-02-28,0.0,Transportes Bello E Hijos Ltda.,39916,88397100.0
3,2019-03-30,0.0,Transportes Bello E Hijos Ltda.,57984,88397100.0
4,2019-04-30,0.0,Transportes Bello E Hijos Ltda.,28992,88397100.0


In [27]:
df_nombres = df_1[['RUT','Cliente']]

In [28]:
df_nombres = df_nombres.groupby(['RUT'])['Cliente'].first().reset_index()

In [29]:
df_nombres.shape

(88971, 2)

In [30]:
df_nombres.loc[df_nombres['RUT'] == 96882280]

,RUT,Cliente
88719,96882280.0,Importadora Transagro S.A.


In [31]:
df_nombres.loc[df_nombres['RUT'] == 74687374]

,RUT,Cliente


In [32]:
df_nombres.to_csv(URL_D + 'df_nombres_para_sinflota.csv', index = False)